In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI

/home/pyrosama/python/agriculture_punjabi_chatbot_capstone/chatbot/chatbot_env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
sentence = ['''ਪੰਜਾਬ ਦੀਆਂ ਖੇਤੀ-ਭੂਗੋਲਿਕ ਹਾਲਾਤਾਂ ਨਿੰਬੂ ਜਾਤੀ ਦੇ ਫ਼ਲਾਂ ਦੇ ਉਤਪਾਦਨ ਲਈ ਸਭ ਤੋਂ
ਵੱਧ ਢੁੱਕਵੀਆਂ ਹਨ। ਦੂਜੇ ਫ਼ਲਾਂ ਦੇ ਮੁਕਾਬਲੇ ਆਪਣੀ ਉੱਚ ਆਰਥਿਕ ਉਤਪਾਦਕਤਾ ਕਾਰਨ
ਨਿੰਬੂ ਜਾਤੀ ਦੇ ਫ਼ਲਾਂ ਦੀ ਖੇਤੀ ਕਿਸਾਨਾਂ ਲਈ ਵਰਦਾਨ ਸਿੱਧ ਹੋਈ ਹੈ। ਇਹ ਫ਼ਲ ਰਕਬਾ ਅਤੇ
ਉਤਪਾਦਨ ਦੋਨਾਂ ਵਿੱਚ ਨੰਬਰ ਇੱਕ ਦੇ ਤੌਰ ਤੇ ਉਭਰੇ ਹਨ ਅਤੇ ਪੰਜਾਬ ਰਾਜ ਵਿਚ “ਸੁਨਹਿਰੀ
ਕ੍ਰਾਂਤੀ” ਲਿਆਏ ਹਨ। ਪਰ ਫੁੱਲ ਪੈਣ ਤੋਂ ਫ਼ਲ ਦੇ ਪੱਕਣ ਸਮੇਂ ਤੱਕ ਫ਼ੁੱਲਾਂ ਅਤੇ ਫ਼ਲਾਂ ਦਾ ਭਾਰੀ
ਮਾਤਰਾ ਵਿੱਚ ਝੜ ਜਾਣਾ ਨਿੰਬੂ ਦੇ ਫ਼ਲਾਂ ਵਿਚ ਇੱਕ ਗੰਭੀਰ ਸਮੱਸਿਆ ਹੈ।''']
# embeddings = model.encode(sentence)

In [4]:
# model_name = 'l3cube-pune/punjabi-sentence-bert-nli'
model_name = 'l3cube-pune/punjabi-sentence-similarity-sbert'
embeddings = HuggingFaceEmbeddings(
    model_name=model_name  # Provide the pre-trained model's path
    # model_kwargs=model_kwargs, # Pass the model configuration options
    # encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/pyrosama/python/agriculture_punjabi_chatbot_capstone/chatbot/chatbot_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/pyrosama/python/agriculture_punjabi_chatbot_capstone/chatbot/chatbot_env/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/pyrosama

In [5]:
f = open('../ocr_code/data_books/ocr_punjabi/pp_fruits_pbi.txt')
sample_doc = f.read()
f.close()

In [6]:
sample_doc = sample_doc.replace('\n\n','#sentencesahikarnekeliyerandomvariable')
sample_doc = sample_doc.replace('\n' ,'')
sample_doc = sample_doc.replace('#sentencesahikarnekeliyerandomvariable','\n\n')

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
splits = text_splitter.create_documents(texts = [sample_doc])

In [8]:
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

In [9]:
retriever = vectorstore.as_retriever(search_kwargs={'k':1})

In [10]:
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             """You are an employ at call center, provide answer to the question from your own knowlage, you may use the provided context
#             context: {context}
#             """,
#         ),
#         ("human", "{question}"),
#     ]
# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ਤੁਸੀਂ ਕਾਲ ਸੈਂਟਰ ਵਿੱਚ ਇੱਕ ਕਰਮਚਾਰੀ ਹੋ, ਆਪਣੇ ਖੁਦ ਦੇ ਗਿਆਨ ਤੋਂ ਸਵਾਲ ਦਾ ਜਵਾਬ ਪ੍ਰਦਾਨ ਕਰੋ, ਤੁਸੀਂ ਪ੍ਰਦਾਨ ਕੀਤੇ ਸੰਦਰਭ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹੋ
            ਸੰਦਰਭ: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)



In [11]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='ਤੁਸੀਂ ਕਾਲ ਸੈਂਟਰ ਵਿੱਚ ਇੱਕ ਕਰਮਚਾਰੀ ਹੋ, ਆਪਣੇ ਖੁਦ ਦੇ ਗਿਆਨ ਤੋਂ ਸਵਾਲ ਦਾ ਜਵਾਬ ਪ੍ਰਦਾਨ ਕਰੋ, ਤੁਸੀਂ ਪ੍ਰਦਾਨ ਕੀਤੇ ਸੰਦਰਭ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹੋ\n            ਸੰਦਰਭ: {context}\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])

In [12]:
llm = ChatMistralAI(
    model="mistral-large-latest"
)

In [13]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
question = 'ਫਲਦਾਰ ਪੌਦਿਆਂ ਲਈ ਕਿਹੜੀ ਜ਼ਮੀਨ ਸਭ ਤੋਂ ਵਧੀਆ ਹੈ'
print(rag_chain.invoke(question))

ਫਲਦਾਰ ਪੌਦਿਆਂ ਲਈ ਸਭ ਤੋਂ ਵਧੀਆ ਜ਼ਮੀਨ ਉਹ ਹੁੰਦੀ ਹੈ ਜਿਸ ਵਿੱਚ ਨਿਰੰਤਰ ਪਾਣੀ ਦੀ ਸਪਲਾਈ, ਸੁਵਿਧਾਜਨਕ ਪਾਣੀ ਦੀ ਨਿਕਾਸੀ ਅਤੇ ਪੌਸ਼ਟਿਕ ਮਿੱਟੀ ਹੋਵੇ। ਫਲਦਾਰ ਪੌਦਿਆਂ ਦੀ ਵਧੀਆ ਵੱਡੀ ਜ਼ਮੀਨ ਦੀਆਂ ਮੁੱਖ ਵਿਸ਼ੇਸ਼ਤਾਵਾਂ ਹਨ:

1. **ਪਾਣੀ ਦੀ ਸਪਲਾਈ**: ਫਲਦਾਰ ਪੌਦਿਆਂ ਨੂੰ ਨਿਰੰਤਰ ਪਾਣੀ ਦੀ ਜ਼ਰੂਰਤ ਹੁੰਦੀ ਹੈ, ਇਸ ਲਈ ਉੱਥੇ ਪਾਣੀ ਦੀ ਸਪਲਾਈ ਸੁਨਿਸਚਿਤ ਹੋਣੀ ਚਾਹੀਦੀ ਹੈ।

2. **ਪਾਣੀ ਦੀ ਨਿਕਾਸੀ**: ਜ਼ਮੀਨ ਵਿੱਚ ਪਾਣੀ ਦੀ ਅਚੁੱਕ ਨਿਕਾਸੀ ਹੋਣੀ ਚਾਹੀਦੀ ਹੈ ਤਾਂ ਕਿ ਜੜ੍ਹਾਂ ਦੀ ਸੜਕ ਨਾ ਹੋਵੇ।

3. **ਪੌਸ਼ਟਿਕ ਮਿੱਟੀ**: ਫਲਦਾਰ ਪੌਦਿਆਂ ਨੂੰ ਪੌਸ਼ਟਿਕ ਮਿੱਟੀ ਦੀ ਜ਼ਰੂਰਤ ਹੁੰਦੀ ਹੈ ਜਿਸ ਵਿੱਚ ਕਾਫ਼ੀ ਖਣਿਜਾਂ ਅਤੇ ਪੌਸ਼ਟਿਕ ਤੱਤ ਹੋਣੇ ਚਾਹੀਦੇ ਹਨ।

4. **ਸੂਰਜ ਦੀ ਰੋਸ਼ਨੀ**: ਫਲਦਾਰ ਪੌਦਿਆਂ ਨੂੰ ਕਾਫ਼ੀ ਸੂਰਜ ਦੀ ਰੋਸ਼ਨੀ ਦੀ ਜ਼ਰੂਰਤ ਹੁੰਦੀ ਹੈ, ਇਸ ਲਈ ਜ਼ਮੀਨ ਉਹ ਹੋਣੀ ਚਾਹੀਦੀ ਹੈ ਜਿੱਥੇ ਸੂਰਜ ਦੀ ਰੋਸ਼ਨੀ ਕਾਫ਼ੀ ਮਾਤਰਾ ਵਿੱਚ ਪਹੁੰਚ ਸਕੇ।

5. **ਮੌਸਮੀ ਹਾਲਾਤ**: ਫਲਦਾਰ ਪੌਦਿਆਂ ਦੇ ਵੱਖ-ਵੱਖ ਕਿਸਮਾਂ ਦੇ ਵੱਖ-ਵੱਖ ਮੌਸਮੀ ਹਾਲਾਤਾਂ ਦੀ ਜ਼ਰੂਰਤ ਹੁੰਦੀ ਹੈ, ਇਸ ਲਈ ਜ਼ਮੀਨ ਦੀ ਚੋਣ ਮੌਸਮੀ ਹਾਲਾਤਾਂ ਨੂੰ ਧਿਆਨ ਵਿੱਚ ਰੱਖਦੇ ਹੋਏ ਕੀਤੀ ਜਾਣੀ ਚਾਹੀਦੀ ਹੈ।

ਇਹਨਾਂ ਤੱਤਾਂ ਦੇ ਸੁਮੇਲ ਨਾਲ ਜ਼ਮੀਨ ਫਲਦਾਰ ਪੌਦਿਆਂ ਲਈ ਸਭ ਤੋਂ ਵਧੀਆ ਹੋਵੇਗੀ।
